In [ ]:
import pandas as pd

In [ ]:
from transformers import XLNetTokenizer
tokenizer = XLNetTokenizer.from_pretrained("/kaggle/input/tokenizer-xlnet")

In [ ]:
import torch
model = torch.load('/kaggle/input/mode-9/model_XLNet.pt')

In [ ]:
data = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')

In [ ]:
from nltk.corpus import stopwords
import re
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    words = text.split() 
    words = [word.lower() for word in words if word.isalpha()] 
    words = [word for word in words if word not in stop_words]  
    return ' '.join(words)

In [ ]:
data['clean_text'] = data['text'].apply(clean_text)


In [ ]:
texts = data['clean_text'] 

In [ ]:
texts = texts.tolist()


In [ ]:
predictions = []

In [ ]:
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
model.to(device)

In [ ]:
model.eval()
with torch.no_grad():
    for text in texts:
        enc = tokenizer(text, truncation=True, padding=True, return_tensors='pt')
        input_ids = enc['input_ids'].to(device)
        attention_mask = enc['attention_mask'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        
        logits = outputs.logits
        logits = F.softmax(logits)
        predictions.append(logits[0][1].item())

In [ ]:
predictions

In [ ]:
sub = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv')


In [ ]:
sub['id'] = data['id']
sub['generated'] = predictions

In [ ]:
sub.to_csv('submission.csv',index=False)